In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from multiprocessing import Pool
import datetime

import scrapers

In [2]:
import itertools


def p_imap_unordered_set(pool, func, iterable):
    temp = pool.imap_unordered(func, iterable)
    return set(itertools.chain.from_iterable(temp))


the_time = lambda: datetime.datetime.now().time().strftime('%H:%M:%S')

In [3]:
riders_done = {"rider/"}
teams_done = {"team/"}
results = set()
teams_to_do = set()

print(the_time())

with Pool(50) as p:
    top_mens_riders = p_imap_unordered_set(
        p, scrapers.scrape_top, itertools.product(range(1946, 2021), range(0, 301, 100), [True])
    )
    top_womens_riders = p_imap_unordered_set(
        p, scrapers.scrape_top, itertools.product(range(2014, 2021), range(0, 301, 100), [False])
    )

top_riders = top_mens_riders | top_womens_riders

riders_to_do = top_riders.copy()
for i in range(50):
    if not riders_to_do:
        break
    print(f"{the_time()} - iter {i+1}, riders_to_do: {len(riders_to_do)}")
    with Pool(50) as p:
        fresh_results = p_imap_unordered_set(p, scrapers.scrape_rider, riders_to_do)

    results.update(fresh_results)
    riders_done.update(riders_to_do)

    teams_to_do = set(team for _, team, _ in fresh_results) - teams_done

    print(f"{the_time()} - teams_to_do: {len(teams_to_do)}")

    with Pool(50) as p:
        riders_to_do = p_imap_unordered_set(p, scrapers.scrape_team, teams_to_do) - riders_done

    teams_done.update(teams_to_do)

print(the_time())

14:14:54
14:15:11 - iter 1, riders_to_do: 7969
14:20:24 - teams_to_do: 5785
14:21:59 - iter 2, riders_to_do: 16013
14:31:39 - teams_to_do: 1284
14:32:05 - iter 3, riders_to_do: 4405
14:35:08 - teams_to_do: 74
14:35:15 - iter 4, riders_to_do: 287
14:35:36 - teams_to_do: 6
14:35:39 - iter 5, riders_to_do: 28
14:35:44 - teams_to_do: 2
14:35:46 - iter 6, riders_to_do: 18
14:35:51 - teams_to_do: 6
14:35:53 - iter 7, riders_to_do: 13
14:35:57 - teams_to_do: 0
14:35:57


In [4]:
with open("top_riders.txt", "w") as f:
    f.write("\n".join(sorted(top_riders)))
    
with open("top_mens_riders.txt", "w") as f:
    f.write("\n".join(sorted(top_mens_riders)))
    
with open("top_womens_riders.txt", "w") as f:
    f.write("\n".join(sorted(top_womens_riders)))

df = pd.DataFrame(iter(results), columns=["rider", "team", "points"])
df.to_csv("scraped.csv", index=False)